In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import twint
import nest_asyncio


%matplotlib inline

In [3]:
nest_asyncio.apply()

In [9]:
t = twint.Config()
t.Username = "YahooSportsNBA"
t.Since = '2021-01-31'
t.Pandas = True
twint.run.Search(t)

1371523338527444993 2021-03-15 14:07:00 -0400 <YahooSportsNBA> The @WNBA unveiled a new logo to celebrate its 25th anniversary 👀 #CountIt   https://t.co/vJwBnTPf7F
1371516350213029896 2021-03-15 13:39:14 -0400 <YahooSportsNBA> 🔘 JaVale McGee getting interest from the Nets 🔘 What's going on with DeMar DeRozan? 🔘 Nurk returning soon?   More notes from @ChrisBHaynes in Hoopla with Haynes ➡️  https://t.co/WyjsDGXVcE  https://t.co/r5yXvEatqR
1371286088405385216 2021-03-14 22:24:15 -0400 <YahooSportsNBA> ANOTHER DAY, ANOTHER ANTHONY EDWARDS JAM 💥   https://t.co/w96pJBWhUT
1371278872881336320 2021-03-14 21:55:35 -0400 <YahooSportsNBA> Jayson Tatum made this look too easy 👀   https://t.co/MyxA5l6b77
1371263992836943874 2021-03-14 20:56:27 -0400 <YahooSportsNBA> Jaylen faked 'em out 😂   https://t.co/MNMHPc0qFA
1371254793176375302 2021-03-14 20:19:54 -0400 <YahooSportsNBA> Ben Simmons with authority 🔨  (via @sixers)   https://t.co/CMlevDT3dv
1371242774528466945 2021-03-14 19:32:08 -0400 <YahooSp

1364892898911686658 2021-02-25 06:00:00 -0400 <YahooSportsNBA> "KD is the best player in the world right now."  @LaJethroJenkins and Seerat Sohi break down the red-hot Nets and which teams could potentially matchup best with them.
1364799134708867073 2021-02-24 23:47:25 -0400 <YahooSportsNBA> Spida too tough 😬  (via @utahjazz)   https://t.co/XNpGEU0ies
1364759455544406021 2021-02-24 21:09:44 -0400 <YahooSportsNBA> Myles says 🚫   https://t.co/6VciNRYd03
1364748956891234304 2021-02-24 20:28:01 -0400 <YahooSportsNBA> Anthony Edwards is sooo mean to the rims 😳   https://t.co/74enz4puOw
1364748302957219843 2021-02-24 20:25:25 -0400 <YahooSportsNBA> Gallo showing off the range 🎯   https://t.co/vPUvwO4Xtn
1364687975511183363 2021-02-24 16:25:42 -0400 <YahooSportsNBA> Fans logging on when they think their team got disrespected on national TV games...  https://t.co/9UwQjJ8A5Q
1364675723395403777 2021-02-24 15:37:01 -0400 <YahooSportsNBA> The NBA released the second half of the 2021 schedule.   

1359224066096443397 2021-02-09 14:34:05 -0400 <YahooSportsNBA> After 11 seasons, @ReneeMontgomery is retiring to focus on her social justice work in Atlanta.  ➡️  https://t.co/1Z63bp8bop  https://t.co/6pP4oAt3ob
1358988145140797443 2021-02-08 22:56:37 -0400 <YahooSportsNBA> Holy Murray 😱   https://t.co/8T4xggYYUO
1358973403214999568 2021-02-08 21:58:02 -0400 <YahooSportsNBA> Anthony Edwards got up 😤   https://t.co/97I4VqQukq
1358944748271140864 2021-02-08 20:04:10 -0400 <YahooSportsNBA> The typically old school Kevin Garnett was unexpectedly complimentary of today’s NBA 👀  ➡️  https://t.co/Tj2ZBB248Q  https://t.co/S25EEzg10v
1358895865566486531 2021-02-08 16:49:56 -0400 <YahooSportsNBA> "Racism is taught."  Donovan Mitchell and Rudy Gobert called out a Utah school for initially allowing parents to opt their students out of Black History Month curriculum.   ➡️  https://t.co/eaOm0nIwSg  https://t.co/et2OyFL6Pv
1358888071119450118 2021-02-08 16:18:57 -0400 <YahooSportsNBA> "You didn't los

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [10]:
df = twint.storage.panda.Tweets_df

In [11]:
df.head()

id      conversation_id    created_at  \
0  1371523338527444993  1371523338527444993  1.615832e+12   
1  1371516350213029896  1371516350213029896  1.615830e+12   
2  1371286088405385216  1371286088405385216  1.615775e+12   
3  1371278872881336320  1371278872881336320  1.615773e+12   
4  1371263992836943874  1371263992836943874  1.615770e+12   

                  date timezone place  \
0  2021-03-15 14:07:00    -0400         
1  2021-03-15 13:39:14    -0400         
2  2021-03-14 22:24:15    -0400         
3  2021-03-14 21:55:35    -0400         
4  2021-03-14 20:56:27    -0400         

                                               tweet language   hashtags  \
0  The @WNBA unveiled a new logo to celebrate its...       en  [countit]   
1  🔘 JaVale McGee getting interest from the Nets ...       en         []   
2  ANOTHER DAY, ANOTHER ANTHONY EDWARDS JAM 💥   h...       en         []   
3  Jayson Tatum made this look too easy 👀   https...       en         []   
4   Jaylen faked 'em out 😂   https://t.co/MNMHPc0qFA       en         []   

  cashtags  ...  geo source user_rt_id user_rt  retweet_id reply_to  \
0       []  ...                                                  []   
1       []  ...                                                  []   
2       []  ...                                                  []   
3       []  ...                                                  []   
4       []  ...                                                  []   

  retweet_date translate trans_src  trans_dest  
0                                               
1                                               
2                                               
3                                               
4                                               

[5 rows x 38 columns]

In [16]:
test_str = df.loc[4,'tweet']
encoded_test_str = test_str.encode('ascii', "ignore")
decoded_test_str = encoded_test_str.decode()
decoded_test_str

"Jaylen faked 'em out    https://t.co/MNMHPc0qFA"

In [19]:
def create_tweets_dataframe(username, start_date, end_date):
    c = twint.Config()
    c.Username = username
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    dataframe = twint.storage.panda.Tweets_df
    return dataframe
    
    
    

In [20]:
yahoo_df = create_tweets_dataframe('YahooSportsNBA', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [22]:
crossover_df = create_tweets_dataframe('TheCrossover', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [23]:
athletic_df = create_tweets_dataframe('TheAthleticNBA', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [24]:
slam_df = create_tweets_dataframe('Slamnewswire', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [25]:
nba_df = create_tweets_dataframe('NBA', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [26]:
bball_news_df = create_tweets_dataframe('basketbllnews', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [35]:
len(yahoo_df) + len(crossover_df) + len(athletic_df) + len(slam_df) + len(nba_df) + len(bball_news_df)

185008

In [37]:
nba_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [ ]:
df_list = [yahoo_df, crossover_df, athletic_df, slam_df, nba_df, bball_news_df]
for dataframe in df_list:
    dataframe.drop(columns=[], inplace=True)